# 02. 🤗 Transformers - 작업별 사용예시

🤗 라이브러리 설치 후 런타임 재시작 필수! 🤗

In [1]:
!pip install transformers datasets accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 963.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

# 1.감성 분석

영화 리뷰, 트윗 등의 텍스트에 대한 감성을 분류 파인튜닝

<img src="https://i.imgur.com/eRLczbU.png" width="700px" />

## 1.1. 모델과 토크나이저 로딩

분류 문제에 맞는 SequenceClassification 모델을 선택

```
BertTokenizerFast
BertForSequenceClassification
```

In [2]:
from transformers import BertForSequenceClassification, BertTokenizerFast

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

다운받은 config.json 파일 내용을 살펴보자


```python
# config.json
{
  "architectures": [ # 모델의 아키텍처를 정의
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1, # Transformer의 attention scores에 적용되는 드롭아웃 확률
  "hidden_act": "gelu", # Transformer 내의 hidden layers에서 사용되는 활성화 함수
  "hidden_size": 768, # 임베딩 벡터의 차원 수
  "intermediate_size": 3072, # Transformer의 feedforward layers의 크기
  "max_position_embeddings": 512, # 모델이 처리할 수 있는 최대 문장 길이
  "num_attention_heads": 12, # Self-attention에서 사용되는 attention heads의 수
  "num_hidden_layers": 12, # hidden layers의 수
  "pad_token_id": 0, # Padding token의 ID
  "vocab_size": 30522, # 모델이 사용하는 어휘 사전의 크기
  ...
}
```

## 1.2. 학습 데이터세트 준비

In [3]:
from datasets import load_dataset

train_dataset = load_dataset('imdb', split="train[:1000]")
train_dataset


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

## 1.3. 훈련 데이터  전처리

토크나이징

In [4]:
def tokenize(batch):
  return tokenizer(
      batch['text'],
      padding=True,
      truncation=True,
      max_length=512, return_tensors='pt'
)



tokenized_dataset = train_dataset.map( tokenize, batched=True, batch_size=512,
                  remove_columns=["text"])

tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [ ]:
tokenized_dataset[0]

## 1.4. 학습 설정 및 훈련

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./bert_fintuned',
    num_train_epochs=1
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=125, training_loss=0.018953563690185547, metrics={'train_runtime': 33.0473, 'train_samples_per_second': 30.26, 'train_steps_per_second': 3.782, 'total_flos': 263111055360000.0, 'train_loss': 0.018953563690185547, 'epoch': 1.0})

## 1.5. 감성분석 모델 추론

In [8]:
inputs = tokenizer("I'm very very happy!", return_tensors='pt').to('cuda')
inputs


{'input_ids': tensor([[ 101, 1045, 1005, 1049, 2200, 2200, 3407,  999,  102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [11]:
# model(input_ids=inputs['input_ids']~~~)
result = model(**inputs).logits # 객체 안의 값들을 풀어서 넣어줌
print(result)

tensor([[ 4.3821, -3.3864]], device='cuda:0', grad_fn=<AddmmBackward0>)


## 1.6. 전체 코드

In [ ]:
# 1) 라이브러리 불러오기
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# 2) 데이터세트 준비
dataset = load_dataset("imdb")
dataset = load_dataset("imdb", split="train[:2500]")


# 3) 모델과 토크나이저 로딩
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


# 4) 훈련 데이터 전처리
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512, return_tensors='pt')

tokenized_dataset = dataset.map(tokenize, batched=True, batch_size=512, remove_columns=["text"])


# 5) 학습 설정 및 훈련
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,                 # Default(3)
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)
trainer.train()

# 6) 추론
inputs = tokenizer("I'm happy.", return_tensors='pt').to('cuda')
result = model(**inputs).logits

# 2.텍스트 생성

<img src="https://i.imgur.com/t7grosm.png" width="700px" />

## 2.1. 모델과 토크나이저 로딩

In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')



config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

다운받은 config.json 파일 내용을 살펴보자

```python
# config.json
{
  "activation_function": "gelu_new", # 히든 레이어에서 사용되는 활성화 함수
  "bos_token_id": 50256, # 문장의 시작을 나타내는 토큰 ID
  "eos_token_id": 50256, # 문장의 끝을 나타내는 토큰 ID
  "n_ctx": 1024, # 모델이 처리할 수 있는 최대 시퀀스 길이
  "n_embd": 768, # 임베딩 벡터의 차원 수
  "n_head": 12, # self-attention에서 사용되는 attention heads의 수
  "n_layer": 12, # 트랜스포머 모델에서 사용되는 층의 수
  "n_positions": 1024, # 입력 포지션의 최대 수
  "vocab_size": 50257, # 모델이 사용하는 어휘 사전의 크기
  ...
}
```

## 2.2. 입력 텍스트 전처리 (토큰화, 인코딩)

In [13]:
input_context = "What is the deep learning?"

input_ids = tokenizer.encode(input_context, return_tensors='pt')
input_ids

tensor([[2061,  318,  262, 2769, 4673,   30]])

## 2.3. 모델 추론 (문장 생성)

In [14]:
output = model.generate(input_ids)
output


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[ 2061,   318,   262,  2769,  4673,    30,   198,   198, 29744,  4673,
           318,   257,   649,  2214,   286,  2267,   326,   468,   587,  1088]])

## 2.4. 결과 텍스트 후처리 (디코딩)

In [15]:
# 4) 결과 출력
for i, output_seq in enumerate(output):
    # 디코딩 코드
    decoded_output = tokenizer.decode(output_seq, skip_special_tokens=True)

    print(f'Generated text {i+1}:')
    print(decoded_output)
    print('\n')





Generated text 1:
What is the deep learning?

Deep learning is a new field of research that has been around




## 2.5. 전체 코드

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 1) 모델과 토크나이저 로딩
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 2) 텍스트 전처리 (토크나이징)
input_context = "What is the Deep learning?"
input_ids = tokenizer.encode(input_context, return_tensors='pt')

# 3) 모델 추론 (문장생성)
output = model.generate(input_ids, max_length=50)

# 4) 결과 출력
for i, output_seq in enumerate(output):
    decoded_output = tokenizer.decode(output_seq, skip_special_tokens=True)

    print(f'Generated text {i+1}:')
    print(decoded_output)
    print('\n')


# 3.질의 응답 시스템

<img src="https://i.imgur.com/0YBTw5J.png" width="700px" />

파이프라인(pipleline) 사용해서 모델 추론하기

In [17]:
from transformers import pipeline


# 참고 문서
context = """
Hugging Face is a leading artificial intelligence research and development company, providing high-performance technologies and solutions, especially in the field of natural language processing (NLP). To make natural language processing tasks easier for a wide range of developers and researchers, Hugging Face develops and shares state-of-the-art deep learning libraries.
Hugging Face's key features include a library called 'transformers'.
"""

question = "What does Hugging Face offer?"


# 질문 응답 파이프라인을 초기화합니다.
qa_pipeline = pipeline(
    'question-answering',
    model='bert-large-uncased-whole-word-masking-finetuned-squad')


# 질문에 대한 답변을 생성합니다.
answer = qa_pipeline({'context':context, 'question':question})


# 답변을 출력합니다.
print(answer)




Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'score': 0.8623039722442627, 'start': 95, 'end': 138, 'answer': 'high-performance technologies and solutions'}
